In [ ]:
# This is for Japan data source #6, Hokkaido Electric Power.
# Contributor(s): Sang-Won Yu

# Japanese fiscal year starts in every April.
# Which means 1st quarter = April, May, June; 2nd quarter = July, August, September etc.
# For Hokkaido, CSVs are quarterly instead of yearly.

# importing necessary modules
import pandas as pd
import numpy as np
from datetime import datetime

def read_hokkaido_csv():
  combined_data = pd.DataFrame()
  for year in range(2016, datetime.now().year + 1): # 2016 - current year
    for quarter in range(1, 5): # 1st quarter - 4th quarter
      try:
        csv = pd.read_csv('https://www.hepco.co.jp/network/renewable_energy/fixedprice_purchase/csv/sup_dem_results_' +
                          str(year) + '_' + str(quarter) + 'q.csv', header = 2,
                          encoding = 'shift_jis')
        csv.drop(labels=0, axis=0, inplace=True) # Delete the empty row at the beginning of the CSV
        combined_data = combined_data.append(csv, ignore_index=True)
      except: # If no such CSV exists yet, skip
        continue

  combined_data = combined_data.replace(np.nan).ffill() # Fill empty trailing values with last known value (for dates)
  combined_data['時刻'] = combined_data['時刻'].str.replace(r'時$',':00') # Fix the format of the time. "時" means "hour".
  combined_data.drop('供給力合計', axis=1, inplace=True) # Delete 供給力合計 column, it's just supply total.

  # combine columns 'DATE' and 'TIME' to make a datetime object
  combined_data['月日']=pd.to_datetime(combined_data['月日'] + ' ' + combined_data['時刻'], format='%Y/%m/%d %H:%M')
  combined_data.drop('時刻', axis=1, inplace=True)

  # Translate Japanese column names to English
  combined_data.columns = ['Date_Time', 'Area_Demand', 'Nuclear', 'Thermal',
                          'Hydraulic', 'Geothermal', 'Biomass', 'Solar(Actual)',
                          'Solar(Output_Control)', 'Wind(Actual)', 'Wind(Output_Control)',
                          'Pumped_Hydro', 'Interconnector']

  # assign units and region
  combined_data['Region'] = 'Hokkaido'
  combined_data['Unit'] = 'MWh'

  # get demand data into one df
  demand_df = combined_data[['Date_Time', 'Region', 'Unit', 'Area_Demand']].copy()
  demand_df.sort_values(by=['Date_Time'],ascending=False, inplace=True)

  # get supply data into another df
  supply_df = combined_data
  supply_df.drop('Area_Demand', axis=1, inplace=True)
  supply_df = pd.melt(combined_data, id_vars=['Date_Time','Region', 'Unit'], var_name='Fuel_Type', value_name='Supply')
  supply_df.sort_values(by=['Date_Time','Fuel_Type'], ascending=False, inplace=True)

  # write df to csvs
  # demand_df.to_csv('Hokkaido_Demand')
  # supply_df.to_csv('Hokkaido_Supply')
  
  return demand_df, supply_df

hokkaido_data = read_hokkaido_csv()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: The default value of regex will change from True to False in a future version.


In [ ]:
hokkaido_data[0]

,Date_Time,Region,Unit,Area_Demand
51123,2022-04-30 23:00:00,Hokkaido,MWh,2792.0
51122,2022-04-30 22:00:00,Hokkaido,MWh,2795.0
51121,2022-04-30 21:00:00,Hokkaido,MWh,2946.0
51120,2022-04-30 20:00:00,Hokkaido,MWh,3091.0
51119,2022-04-30 19:00:00,Hokkaido,MWh,3229.0
...,...,...,...,...
4,2016-04-01 04:00:00,Hokkaido,MWh,3598.0
3,2016-04-01 03:00:00,Hokkaido,MWh,3468.0
2,2016-04-01 02:00:00,Hokkaido,MWh,3387.0
1,2016-04-01 01:00:00,Hokkaido,MWh,3282.0


In [ ]:
hokkaido_data[1]

,Date_Time,Region,Unit,Fuel_Type,Supply
460115,2022-04-30 23:00:00,Hokkaido,MWh,Wind(Output_Control),0.0
408991,2022-04-30 23:00:00,Hokkaido,MWh,Wind(Actual),32.0
102247,2022-04-30 23:00:00,Hokkaido,MWh,Thermal,2520.0
357867,2022-04-30 23:00:00,Hokkaido,MWh,Solar(Output_Control),0.0
306743,2022-04-30 23:00:00,Hokkaido,MWh,Solar(Actual),0.0
...,...,...,...,...,...
0,2016-04-01 00:00:00,Hokkaido,MWh,Nuclear,0.0
511240,2016-04-01 00:00:00,Hokkaido,MWh,Interconnector,42.0
102248,2016-04-01 00:00:00,Hokkaido,MWh,Hydraulic,474.0
153372,2016-04-01 00:00:00,Hokkaido,MWh,Geothermal,13.0
